# Conférences Python Master TIDE #5

1. **Web scraping**
2. **API**

&copy; 2025 Francis Wolinski

## 0. Rappels

1. **Protocoles HTTP / HTTPS**

- **HTTP** : HyperText Transfer Protocol, protocole permettant au client (navigateur / script) de communiquer avec un serveur web.
- **HTTPS** : HyperText Transfer Protocol Secure, version sécurisée de HTTP, chiffrée via TLS (Transport Layer Security).

Il existe plusieurs méthodes dont :
- *GET* : envoie une requête pour obtenir une ressource. Les paramètres sont dans l’URL.
- *POST* : envoie des données dans le corps de la requête (formulaires, connexions), plus adapté pour transmettre des informations sensibles ou volumineuses.

Les **entêtes HTTP** (HTTP headers) sont des informations supplémentaires envoyées avec chaque requête et réponse HTTP. Elles décrivent le contexte de la communication : type de contenu attendu ou envoyé (`Content-Type`), informations sur le client (`User-Agent`), gestion du cache (`Cache-Control`), cookies, authentification, encodage, etc. Pour le web scraping, modifier certains entêtes — par exemple le `User-Agent` — permet de se faire reconnaître comme un navigateur classique, tandis qu’en lire d’autres aide à comprendre comment le serveur gère la session ou les données retournées.

2. **HTML et balises de base**

**HTML** : HyperText Markup Language, langage qui structure le contenu d’une page web :

- `<p>…</p>` : paragraphe de texte.
- `<br />` : retour à la ligne (balise autofermante).

Ces balises constituent la structure que le web scraper va analyser pour extraire des données.

3. **CSS**

**CSS** : Cascading Style Sheets

- *id* : identifiant unique d’un élément dans une page, utile pour cibler précisément un élément,
- *class* : groupe d’éléments partageant le même style ou comportement),
- *style* : définition directe d’un style sur l’élément.

Exemple:

```html
<p id="1234", class="class1 class2", style="color:blue;font-size:20px;">Exemple de paragraphe.</p>
```

En scraping, les tags et les attributs *id* et *class* servent souvent à sélectionner les bons éléments dans le DOM.

<p id="1234", class="class1 class2", style="color:blue;font-size:20px;">Exemple de paragraphe.</p>

## 1. Web Scraping

- Extraction d'informations d'un site web.
- A utiliser en l'absence de données ouvertes ou d'API.
- Technique fragile car le site web peut changer du jour au lendemain.
- Problématique juridique...

**Avec requests et Beautiful Soup**

Doc :
- **requests** : https://requests.readthedocs.io/en/master/
- **Beautiful Soup** : https://beautiful-soup-4.readthedocs.io/en/latest/

Pages de test :
- https://yotta-conseil.fr/cours/page2.html
- https://yotta-conseil.fr/cours/page3.html

Miroir du site : http://kim.fspot.org/

In [ ]:
import requests

r = requests.get('https://yotta-conseil.fr/cours/page2.html')
r.status_code

In [ ]:
# type
type(r.content)

In [ ]:
# str en précisant un encodage
content = r.content.decode('utf-8')
type(content)

In [ ]:
# BeautifulSoup

from bs4 import BeautifulSoup

soup = BeautifulSoup(content, "html.parser")
soup

Accès au texte brut

In [ ]:
# accès au texte brut
soup.get_text()

Pour extraire des éléments particuliers, on utilise différentes méthodes :
    
- `.find(tag, attrs)` : trouve le premier tag avec les attributs spécifiés (en général *id* et/ou *class*)
- `.findAll(tag, attrs)` : trouve tous les tags avec les attributs spécifiés (en général *id* et/ou *class*)
- `.select(css)` : trouve tous les tags avec les CSS spécifiées
- etc.

La technique consiste par exemple à alimenter un dictionnaires avec les valeurs trouvées :
- soit en utilisant `tag.attrs['attribut']` pour collecter la valeur `'attribut'` du tag
- soit en utilisant `tag.text` pour collecter la valeur `text` située entre les balises ouvrante et fermante,
- on peut aussi utiliser d'autres méthodes comme `.select()` qui joue sur la CSS ou `.next_sibling()` pour collecter le tag identique suivant,
- éventuellement en recherchant dans un nouveau tag à l'intérieur d'un tag donné
- parfois, il faut tester si la méthode `.find()` retourne bien un élément ou `None`.

In [ ]:
# exploration du HTML

div_tag = soup.find('div', attrs={'class': 'pricing-table'})
div_tag

Méthode :
 1. On collecte les informations d'un  tag pour un seul enregistrement sous la forme d'un dictionaire.
 2. On généralise en bouclant sur tous les tags de même nature et on rassemble les dictionnaires dans une liste.
 3. On transforme la liste de dictionaires en DataFrame.

In [ ]:
# 1. collecte des informations pour un enregistrement
row = {}

div_tag = soup.find('div', attrs={'class': 'pricing-table'})

# type : Personal, Small Business, Enterprise
h2_tag = div_tag.find("h2")
row["type"] = h2_tag.text

# price
span_tag = div_tag.find("span", attrs={'class': 'pricing-table-price'})
row["price"] = span_tag.text.strip().split()[0]

# storage, database
lis = div_tag.select("li")
row["storage"] = lis[3].text.split()[0]
row["database"] = lis[4].text.split()[0]

row

In [ ]:
# 2. collecte des informations pour tous les enregistrements
rows = []

for div_tag in soup.find_all('div', attrs={'class': 'pricing-table'}):
    row = {}
    
    # type : Personal, Small Business, Enterprise
    h2_tag = div_tag.find("h2")
    row["type"] = h2_tag.text
    
    # price
    span_tag = div_tag.find("span", attrs={'class': 'pricing-table-price'})
    row["price"] = span_tag.text.strip().split()[0]
    
    # storage, database
    lis = div_tag.select("li")
    row["storage"] = lis[3].text.split()[0]
    row["database"] = lis[4].text.split()[0]
    
    rows.append(row)

rows

In [ ]:
# 3. transformation en DataFrame
import pandas as pd
df = pd.DataFrame(rows)
df

In [ ]:
# avec une fonction
def get_product_info(div_tag):
    row = {}

    # type : Personal, Small Business, Enterprise
    h2_tag = div_tag.find("h2")
    row["type"] = h2_tag.text

    # price
    span_tag = div_tag.find("span", attrs={'class': 'pricing-table-price'})
    row["price"] = span_tag.text.strip().split()[0]

    # storage, database
    lis = div_tag.select("li")
    row["storage"] = lis[3].text.split()[0]
    row["database"] = lis[4].text.split()[0]
    
    return row

def extract_products(url):
    try:
        r = requests.get('https://yotta-conseil.fr/cours/page2.html')
        
    except requests.exceptions.RequestException as e:
        print(e)
        return

    # Soup
    soup = BeautifulSoup(r.content, "html.parser")

    # list comprehension with all results
    rows = [get_product_info(div_tag) for div_tag in soup.find_all('div', attrs={'class': 'pricing-table'})]

    # DataFrame with all results
    df = pd.DataFrame(rows)
    
    return df

df = extract_products(url)
df

**Exercices**

- Remplacer la méthode `.find_all()` par la méthode `.select()`
- Web scrapping de la page : https://yotta-conseil.fr/cours/page3.html
- Web scrapping de la page : https://books.toscrape.com/catalogue/category/books/travel_2/index.html

#### Inconvénients du web scraping:
- plutôt lent (car on parse potentiellement beaucoup de HTML inutile)
- ne donne pas les résultats attendus si une partie du contenu est intégré dynamiquement à la page via javascript
- un changement dans l'architecture du html ou du css (e.g: refonte du design du site) oblige à réécrire entièrement le programme

## 2. API

Exemple : Deezer

Artiste : https://www.deezer.com/fr/artist/3037

Récupérer le nombre de fans d'un artiste avec requests (cherchez le tag div avec l'id `"naboo_artist_social_small"`).

In [ ]:
# request
artist = 3037
response = requests.get(f'https://www.deezer.com/fr/artist/{artist}')
print(f"{len(response.content):,.0f} bytes")

In [ ]:
soup = BeautifulSoup(response.content)
n = soup.find("div", attrs={"id": "naboo_artist_social_small"}).find("span").text
print(f"{int(n):,.0f} fans")

Le terme "API" est très générique et peut désigner bien des choses, mais dans le jargon on l'utilise souvent pour désigner un service web qui renvoie non pas:
> des pages web au format HTML (destinées à être lues par un humain dans son navigateur)

mais:
> des données au format JSON (ou texte, ou XML, destinées à être traitées par un programme)

![img](https://miro.medium.com/max/4238/1*OcmVkcsM5BWRHrg8GC17iw.png)

Puisque les API sont dédiées à l'usage via des programmes, elles disposent en général d'une bonne documentation, et sont fiables et stables dans le temps. Tandis que sur des pages web HTML classiques, le design peut par exemple changer du jour au lendemain et rendre votre programme BeautifulSoup obsolète.

In [ ]:
# API JSON
response = requests.get(f'https://api.deezer.com/artist/{artist}')
response.content

In [ ]:
# json
data = response.json()
data

In [ ]:
# nb_fan
print(f"{data['nb_fan']:,.0f} fans")

In [ ]:
# picture
from IPython import display

url = data['picture']
r = requests.get(url)
display.Image(data=r.content)

In [ ]:
# tracklist
url = data["tracklist"]
r = requests.get(url)
dico = r.json()
dico["data"][0]

**Exercice**

- Récupérer la liste des titres avec une list comprehension,
- Jouer un morceau à partir du titre avec `IPython.display.Video()`

#### Avantages d'une API
- renvoie du format JSON, (ou texte ou XML), facile et rapide à traiter
- renvoie un format stable et documenté (voire versionné)
- exemple : https://github.com/bluesky-social/atproto
- la documentation indique comment interagir avec l'API:
    - quelle URL utiliser,
    - quelle méthode HTTP (GET, POST, ...),
    - quels paramètres utiliser,
    - ...

=> idéal pour les développeurs

### Quel intérêt pour le fournisseur d'API ?

En général il met en place des quotas de requêtes ou d'autres limitations afin de proposer un service payant qui dispose de possibilités avancées / d'un meilleur support / etc.

C'est pourquoi de nombreux services nécessitent de se connecter avec son compte client pour utiliser une API (e.g. https://openweathermap.org/api)

#### Basic Auth

Exemple: accéder à https://yotta-conseil.fr/private/

Pour y accéder il est nécessaire d'utiliser les credentials suivant:
- *login* : `'admin'`
- *password* : `'secret'`

Si on ne les passe pas (ou si on ne passe pas les bons), on a obtient une erreur 401 (= unauthorized).

In [ ]:
# sans login/password
res = requests.get('https://yotta-conseil.fr/private')
res

In [ ]:
# avec login password
res = requests.get('https://yotta-conseil.fr/private', auth=('admin', 'secret'))
res

In [ ]:
# contenu
res.content.decode('utf-8')

<p style="color:red;">Attention, ne pas mettre les credentials dans le code !</p>

Exemple avec la librairie **dotenv**.

```bash
pip install python-dotenv
```

La librairie peut gérer les variables d'environnement et également parser des fichiers de type `.env`.

```txt
username=admin
password=secret
```

In [ ]:
#!pip install python-dotenv

In [ ]:
# .env pour masquer auth credentials
# fichier .env avec les credentials

from dotenv import dotenv_values

config = dotenv_values(".env0")

requests.get('https://yotta-conseil.fr/private', auth=(config['username'], config['password']))

#### Auth par token

Exemple sur openweathermap :
- documentation: https://openweathermap.org/appid
- mes tokens: https://home.openweathermap.org/api_keys

In [ ]:
# requête avec un token
config = dotenv_values(".env")
token = config["owm_token"]
n = 4
print(f"{token[:n]}{"x"*(len(token)-2*n)}{token[-n:]}")

Avantage des tokens :
- évite que les requêtes HTTP contiennent le mot de passe - à la place elles contiennent un token
- si je me fais "voler" un token, je peux le supprimer de mon compte et en créer un nouveau

In [ ]:
# requête avec un token
ville="Paris"
url = f"http://api.openweathermap.org/data/2.5/weather?APPID={token}&q={ville}"
res = requests.get(url)
meteo = res.json()
meteo

In [ ]:
# type
type(meteo)

In [ ]:
# extractions
{'city': meteo['name'],
'country': meteo['sys']['country'],
'date': meteo['dt'],
'temp': meteo['main']['temp']}

In [ ]:
# extractions
import time

{'city': meteo['name'],
'country': meteo['sys']['country'],
'date': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(meteo['dt'])),
'temp': f"{meteo['main']['temp'] - 273.15:.1f}",}

**Exercice**

Se créer un compte sur **BlueSky** et requêter des posts. Nécessite d'installer la librairie **atproto**.

Doc : https://atproto.blue/en/latest/atproto/atproto_client.models.app.bsky.feed.search_posts.html

```python
import atproto

# get credentials
config = dotenv_values(".env")

# atproto client
client = atproto.Client()
client.login(config['username_bsky'], config['password_bsky'])

# réponse limitée à 25 posts par défaut, maximum = 100, ensuite il faut gérer un curseur
response = client.app.bsky.feed.search_posts(q="...", ...)
```

Liste des posts : `response.posts`

Pour chaque post :
- `post.record` (`.text`, `.created_at`)
- `post.author` (`.handle`)
- `post.uri`
- `post.*_count` (engagements : bookmark, like, quote, reply, repost)

Ensuite, possibilité de faire du NLP sur les textes des posts.

```python
# réponse limitée également, ensuite il faut gérer un curseur
response = client.app.bsky.graph.get_followers(actor="...", ...)
```

Ensuite, possibilité de faire des graphes sur les followers.